In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text

DB_NAME ="PublicTransit"
DB_USER= "postgres"
DB_PASS="L745tyZxvc."
DB_HOST="localhost"
DB_PORT="5432"

In [2]:
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")
conn =engine.connect()

sql_query="""
create table if not exists monthly_totals
(
id serial,
month varchar,
year int,
city varchar,
transportation_type varchar,
ridership_count int
);"""

In [3]:
ctadf=pd.read_csv('CTA_-_Ridership_-_Daily_Boarding_Totals_20240307.csv')
display(ctadf)

,service_date,day_type,bus,rail_boardings,total_rides
0,01/01/2001,U,297192,126455,423647
1,01/02/2001,W,780827,501952,1282779
2,01/03/2001,W,824923,536432,1361355
3,01/04/2001,W,870021,550011,1420032
4,01/05/2001,W,890426,557917,1448343
...,...,...,...,...,...
8396,10/27/2023,W,503840,373004,876844
8397,10/28/2023,A,360484,275006,635490
8398,10/29/2023,U,250473,185823,436296
8399,10/30/2023,W,551840,367453,919293


In [4]:
mtadf=pd.read_csv('MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv')
display(mtadf)

,Month,Agency,Ridership
0,01/01/2008,MNR,6453734
1,02/01/2008,MNR,6144377
2,03/01/2008,MNR,6602280
3,04/01/2008,MNR,6787480
4,04/01/2008,B&T,24900328
...,...,...,...
643,01/01/2024,LIRR,5490105
644,01/01/2024,AAR,928328
645,01/01/2024,SIR,189970
646,01/01/2024,Bus,24912126


In [5]:
display(ctadf['service_date'].value_counts())
display(mtadf['Agency'].value_counts())

service_date
10/20/2011    2
10/13/2011    2
10/19/2011    2
10/18/2011    2
10/17/2011    2
             ..
07/26/2008    1
07/25/2008    1
07/24/2008    1
07/23/2008    1
10/31/2023    1
Name: count, Length: 8339, dtype: int64

Agency
B&T       190
MNR       180
Bus        73
Subway     73
AAR        61
LIRR       58
SIR        13
Name: count, dtype: int64

In [6]:
mtadf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Month      648 non-null    object
 1   Agency     648 non-null    object
 2   Ridership  648 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 15.3+ KB


In [7]:
ctadf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8401 entries, 0 to 8400
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   service_date    8401 non-null   object
 1   day_type        8401 non-null   object
 2   bus             8401 non-null   int64 
 3   rail_boardings  8401 non-null   int64 
 4   total_rides     8401 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 328.3+ KB


In [8]:
ctadf["service_date"]=pd.to_datetime(ctadf["service_date"])
ctadf.head()

,service_date,day_type,bus,rail_boardings,total_rides
0,2001-01-01,U,297192,126455,423647
1,2001-01-02,W,780827,501952,1282779
2,2001-01-03,W,824923,536432,1361355
3,2001-01-04,W,870021,550011,1420032
4,2001-01-05,W,890426,557917,1448343


In [9]:
ctadf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8401 entries, 0 to 8400
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   service_date    8401 non-null   datetime64[ns]
 1   day_type        8401 non-null   object        
 2   bus             8401 non-null   int64         
 3   rail_boardings  8401 non-null   int64         
 4   total_rides     8401 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 328.3+ KB


In [10]:
mtadf["Month"]=pd.to_datetime(mtadf["Month"])
mtadf.head()

,Month,Agency,Ridership
0,2008-01-01,MNR,6453734
1,2008-02-01,MNR,6144377
2,2008-03-01,MNR,6602280
3,2008-04-01,MNR,6787480
4,2008-04-01,B&T,24900328


In [11]:
ctadfclean=ctadf.drop(columns = ["day_type","total_rides"])
ctadfclean.head()

,service_date,bus,rail_boardings
0,2001-01-01,297192,126455
1,2001-01-02,780827,501952
2,2001-01-03,824923,536432
3,2001-01-04,870021,550011
4,2001-01-05,890426,557917


In [12]:
ctamelt=pd.melt(ctadfclean, id_vars=['service_date'],value_vars = ['bus','rail_boardings'], var_name = 'transportation_type',value_name='ridership count' )
ctamelt.head()

,service_date,transportation_type,ridership count
0,2001-01-01,bus,297192
1,2001-01-02,bus,780827
2,2001-01-03,bus,824923
3,2001-01-04,bus,870021
4,2001-01-05,bus,890426


In [13]:
ctamelt['city']= 'Chicago'
ctamelt.head()

,service_date,transportation_type,ridership count,city
0,2001-01-01,bus,297192,Chicago
1,2001-01-02,bus,780827,Chicago
2,2001-01-03,bus,824923,Chicago
3,2001-01-04,bus,870021,Chicago
4,2001-01-05,bus,890426,Chicago


In [14]:
def find_month(date):
    if date.month == 1:
        return 'January'
    elif date.month == 2:
        return 'February'
    elif date.month == 3:
        return 'March'
    elif date.month == 4:
        return 'April'
    elif date.month == 5:
        return 'May'
    elif date.month == 6:
        return 'June'
    elif date.month == 7:
        return 'July'
    elif date.month == 8:
        return 'August'
    elif date.month == 9:
        return 'September'
    elif date.month == 10:
        return 'October'
    elif date.month == 11:
        return 'November'
    elif date.month == 12:
        return 'December'
    else:
        return 'Invalid month'

ctamelt['month']= ctamelt['service_date'].apply(find_month)

In [15]:
ctamelt.head()

,service_date,transportation_type,ridership count,city,month
0,2001-01-01,bus,297192,Chicago,January
1,2001-01-02,bus,780827,Chicago,January
2,2001-01-03,bus,824923,Chicago,January
3,2001-01-04,bus,870021,Chicago,January
4,2001-01-05,bus,890426,Chicago,January


In [16]:
ctamelt['year'] = ctamelt['service_date'].apply(lambda date: date.year)
ctamelt.head()

,service_date,transportation_type,ridership count,city,month,year
0,2001-01-01,bus,297192,Chicago,January,2001
1,2001-01-02,bus,780827,Chicago,January,2001
2,2001-01-03,bus,824923,Chicago,January,2001
3,2001-01-04,bus,870021,Chicago,January,2001
4,2001-01-05,bus,890426,Chicago,January,2001


In [17]:
ctafinal=ctamelt.drop(columns=["service_date"])

In [18]:
ctafinal=ctafinal.groupby(['year','month','transportation_type','city']).sum().reset_index()
ctafinal.head()

,year,month,transportation_type,city,ridership count
0,2001,April,bus,Chicago,25053791
1,2001,April,rail_boardings,Chicago,14713336
2,2001,August,bus,Chicago,24647188
3,2001,August,rail_boardings,Chicago,16014736
4,2001,December,bus,Chicago,23428582


In [19]:
mtadf['month']= mtadf['Month'].apply(find_month)
mtadf.head()

,Month,Agency,Ridership,month
0,2008-01-01,MNR,6453734,January
1,2008-02-01,MNR,6144377,February
2,2008-03-01,MNR,6602280,March
3,2008-04-01,MNR,6787480,April
4,2008-04-01,B&T,24900328,April


In [20]:
mtadf['year'] = mtadf['Month'].apply(lambda date: date.year)
mtadf.head()

,Month,Agency,Ridership,month,year
0,2008-01-01,MNR,6453734,January,2008
1,2008-02-01,MNR,6144377,February,2008
2,2008-03-01,MNR,6602280,March,2008
3,2008-04-01,MNR,6787480,April,2008
4,2008-04-01,B&T,24900328,April,2008


In [21]:
mtadf=mtadf.rename(columns={"Ridership":"ridership_count"})

In [22]:
def transporttype(agency):
    if agency=="MNR" or agency== "Subway" or agency == "LIRR" or agency=="SIR":
        return "metro"
    else:
        return "bus"     

mtadf= mtadf[mtadf['Agency']!='B&T']

In [23]:
mtadf['transportation_type']=mtadf['Agency'].apply(transporttype)

In [24]:
finmtadf=mtadf.groupby(['transportation_type','year','month'], as_index=False)['ridership_count'].sum().reset_index()
display(finmtadf)

,index,transportation_type,year,month,ridership_count
0,0,bus,2018,April,48544481
1,1,bus,2018,August,46810522
2,2,bus,2018,December,44033258
3,3,bus,2018,February,44719811
4,4,bus,2018,January,45398837
...,...,...,...,...,...
261,261,metro,2023,May,110376738
262,262,metro,2023,November,104558133
263,263,metro,2023,October,110197921
264,264,metro,2023,September,101009689


In [25]:
finalmtadf = mtadf.drop(columns=["Month", "Agency"])
finalmtadf = finalmtadf.groupby(['year', 'month', 'transportation_type']).head()
finalmtadf.head()

,ridership_count,month,year,transportation_type
0,6453734,January,2008,metro
1,6144377,February,2008,metro
2,6602280,March,2008,metro
3,6787480,April,2008,metro
5,6840901,May,2008,metro


In [26]:
combineddf=pd.concat([ctafinal,finalmtadf])
display(combineddf)

,year,month,transportation_type,city,ridership count,ridership_count
0,2001,April,bus,Chicago,25053791.0,NaN
1,2001,April,rail_boardings,Chicago,14713336.0,NaN
2,2001,August,bus,Chicago,24647188.0,NaN
3,2001,August,rail_boardings,Chicago,16014736.0,NaN
4,2001,December,bus,Chicago,23428582.0,NaN
...,...,...,...,...,...,...
642,2024,January,metro,NaN,NaN,91419551.0
643,2024,January,metro,NaN,NaN,5490105.0
644,2024,January,bus,NaN,NaN,928328.0
645,2024,January,metro,NaN,NaN,189970.0


In [27]:
for index, row in combineddf.iterrows():
    values= tuple(row)
    query =f"insert into monthly_totals(year,month, transportation_type, city, ridership_count)values {values}"
    conn.execute(query)

ObjectNotExecutableError: Not an executable object: "insert into monthly_totals(year,month, transportation_type, city, ridership_count)values (2001, 'April', 'bus', 'Chicago', 25053791.0, nan)"